## 相关源码
```
framework/base/core/java/andorid/os/MessageQueue.java
framework/base/core/jni/android_os_MessageQueue.cpp
framework/base/core/java/andorid/os/Looper.java

system/core/libutils/Looper.cpp
system/core/include/utils/Looper.h
system/core/libutils/RefBase.cpp

framework/base/native/android/looper.cpp
framework/native/include/android/looper.h
```

---

## 1. 概述
- 除了 MessageQueue 的 native 方法，native 层本身也有一套完整的消息机制，用于处理 native 的消息，如下图 Native 层的消息机制
- Java 层可以向 MessageQueue 消息队列中添加消息，Native 层也可以向 MessageQueue 消息队列中添加消息

![image](handler2_page1.png)

---

## 2. MessageQueue
```
private native static long nativeInit();
private native static void nativeDestroy(long ptr);
private native void nativePollOnce(long ptr, int timeoutMillis);
private native static void nativeWake(long ptr);
private native static boolean nativeIsPolling(long ptr);
private native static void nativeSetFileDescriptorEvents(long ptr, int fd, int events);
```

### 2.1 nativeInit()
![image](handler2_page2.png)

#### android_os_MessageQueue_nativeInit()

```
static jlong android_os_MessageQueue_nativeInit(JNIEnv* env, jclass clazz) {
    //初始化native消息队列 【3】
    NativeMessageQueue* nativeMessageQueue = new NativeMessageQueue(); 
    nativeMessageQueue->incStrong(env); //增加引用计数
    return reinterpret_cast<jlong>(nativeMessageQueue);
}
```

#### new NativeMessageQueue()
- Looper::getForThread()，功能类比于 Java 层的 Looper.myLooper();
- Looper::setForThread(mLooper)，功能类比于 Java 层的 ThreadLocal.set();


```
NativeMessageQueue::NativeMessageQueue() 
            : mPollEnv(NULL), mPollObj(NULL), mExceptionObj(NULL) {
    mLooper = Looper::getForThread();   // 获取 TLS 中的 Looper 对象
    if (mLooper == NULL) {
        mLooper = new Looper(false);    // 创建 native 层的 Looper 【4】
        Looper::setForThread(mLooper);  // 保存 native 层的 Looper 到 TLS
    }
}
```

[参考文档：epoll原理](http://gityuan.com/2015/12/06/linux_epoll/)

---

### 2.2 nativeDestroy()
![image](handler2_page2.png)

#### MessageQueue.dispose()
```
private void dispose() {
    if (mPtr != 0) {
        nativeDestroy(mPtr); 【2】
        mPtr = 0;
    }
}
```

#### android_os_MessageQueue_nativeDestroy()
```
static void android_os_MessageQueue_nativeDestroy(JNIEnv* env, jclass clazz, jlong ptr) {
    NativeMessageQueue* nativeMessageQueue = reinterpret_cast<NativeMessageQueue*>(ptr);
    nativeMessageQueue->decStrong(env); 【3】
}
```

#### RefBase::decStrong()
```
void RefBase::decStrong(const void* id) const {
    weakref_impl* const refs = mRefs;
    refs->removeStrongRef(id); // 移除强引用
    const int32_t c = android_atomic_dec(&refs->mStrong);
    if (c == 1) {
        refs->mBase->onLastStrongRef(id);
        if ((refs->mFlags&OBJECT_LIFETIME_MASK) == OBJECT_LIFETIME_STRONG) {
            delete this;
        }
    }
    refs->decWeak(id);  // 移除弱引用
}
```

---

### 2.3 nativePollOnce()
![image](handler2_page4.png)

#### MessageQueue.next()
```
Message next() {
    final long ptr = mPtr;
    if (ptr == 0) {
        return null;
    }

    for (;;) {
        ...
        nativePollOnce(ptr, nextPollTimeoutMillis); // 阻塞操作 【2】
        ...
    }
}
```

#### android_os_MessageQueue_nativePollOnce()
```
static void android_os_MessageQueue_nativePollOnce(JNIEnv* env, jobject obj, jlong ptr, jint timeoutMillis) {
    // 将 Java 层传递下来的 mPtr 转换为 nativeMessageQueue
    NativeMessageQueue* nativeMessageQueue = reinterpret_cast<NativeMessageQueue*>(ptr);
    nativeMessageQueue->pollOnce(env, obj, timeoutMillis); 【3】
}
```

#### NativeMessageQueue::pollOnce()
```
void NativeMessageQueue::pollOnce(JNIEnv* env, jobject pollObj, int timeoutMillis) {
    mPollEnv = env;
    mPollObj = pollObj;
    mLooper->pollOnce(timeoutMillis); 【4】
    mPollObj = NULL;
    mPollEnv = NULL;
    if (mExceptionObj) {
        env->Throw(mExceptionObj);
        env->DeleteLocalRef(mExceptionObj);
        mExceptionObj = NULL;
    }
}
```

#### Looper::pollOnce()
```
inline int pollOnce(int timeoutMillis) {
    return pollOnce(timeoutMillis, NULL, NULL, NULL); 【5】
}
```

#### Looper::pollOnce()
- 参数说明
  - timeoutMillis：超时时长
  - outFd：发生事件的文件描述符
  - outEvents：当前outFd上发生的事件，包含以下4类事件
    - EVENT_INPUT 可读
    - EVENT_OUTPUT 可写
    - EVENT_ERROR 错误
    - EVENT_HANGUP 中断
  - outData：上下文数据


```
int Looper::pollOnce(int timeoutMillis, int* outFd, int* outEvents, void** outData) {
    int result = 0;
    for (;;) {
        // 先处理没有 Callback 方法的 Response 事件
        while (mResponseIndex < mResponses.size()) {
            const Response& response = mResponses.itemAt(mResponseIndex++);
            int ident = response.request.ident;
            if (ident >= 0) {
                // ident 大于 0，则表示没有 callback, 因为 POLL_CALLBACK = -2,
                int fd = response.request.fd;
                int events = response.events;
                void* data = response.request.data;
                if (outFd != NULL) *outFd = fd;
                if (outEvents != NULL) *outEvents = events;
                if (outData != NULL) *outData = data;
                return ident;
            }
        }
        if (result != 0) {
            if (outFd != NULL) *outFd = 0;
            if (outEvents != NULL) *outEvents = 0;
            if (outData != NULL) *outData = NULL;
            return result;
        }
        // 再处理内部轮询
        result = pollInner(timeoutMillis); 【6】
    }
}
```

### poll 小结
- 先调用 epoll_wait()，这是阻塞方法，用于等待事件发生或者超时;
- 对于 epoll_wait() 返回，当且仅当以下3种情况出现：
  - POLL_ERROR，发生错误，直接跳转到 Done
  - POLL_TIMEOUT，发生超时，直接跳转到 Done
  - 检测到管道有事件发生，则再根据情况做相应处理：
    - 如果是管道读端产生事件，则直接读取管道的数据
    - 如果是其他事件，则处理 request，生成对应的 reponse 对象，push 到 reponse 数组
- 进入 Done 标记位的代码段
  - 先处理 Native 的 Message，调用 Native 的 Handler 来处理该 Message
  - 再处理 Response 数组，POLL_CALLBACK 类型的事件
  
---

### 2.4 nativeWake()
- nativeWake 用于唤醒功能，在添加消息到消息队列 enqueueMessage(), 或者把消息从消息队列中全部移除 quit()，再有需要时都会调用 nativeWake 方法

![image](handler2_page5.png)

#### android_os_MessageQueue_nativeWake()
```
static void android_os_MessageQueue_nativeWake(JNIEnv* env, jclass clazz, jlong ptr) {
    NativeMessageQueue* nativeMessageQueue = reinterpret_cast<NativeMessageQueue*>(ptr);
    nativeMessageQueue->wake(); 【3】
}
```

#### NativeMessageQueue::wake()
```
void NativeMessageQueue::wake() {
    mLooper->wake();  【4】
}
```

#### Looper::wake()
- TEMP_FAILURE_RETRY 是一个宏定义， 当执行 write 失败后，会不断重复执行，直到执行成功为止

```
void Looper::wake() {
    uint64_t inc = 1;
    // 向管道 mWakeEventFd 写入字符1
    ssize_t nWrite = TEMP_FAILURE_RETRY(write(mWakeEventFd, &inc, sizeof(uint64_t)));
    if (nWrite != sizeof(uint64_t)) {
        if (errno != EAGAIN) {
            ALOGW("Could not write wake signal, errno=%d", errno);
        }
    }
}
```

---

### 2.5 sendMessage
#### sendMessage
```
void Looper::sendMessage(const sp<MessageHandler>& handler, const Message& message) {
    nsecs_t now = systemTime(SYSTEM_TIME_MONOTONIC);
    sendMessageAtTime(now, handler, message);
}
```

#### sendMessageDelayed
```
void Looper::sendMessageDelayed(nsecs_t uptimeDelay, const sp<MessageHandler>& handler,
        const Message& message) {
    nsecs_t now = systemTime(SYSTEM_TIME_MONOTONIC);
    sendMessageAtTime(now + uptimeDelay, handler, message);
}
```

#### sendMessageAtTime
```
void Looper::sendMessageAtTime(nsecs_t uptime, const sp<MessageHandler>& handler,
        const Message& message) {
    size_t i = 0;
    { 
        // 请求锁
        AutoMutex _l(mLock);
        size_t messageCount = mMessageEnvelopes.size();
        // 找到 message 应该插入的位置i
        while (i < messageCount && uptime >= mMessageEnvelopes.itemAt(i).uptime) {
            i += 1;
        }
        MessageEnvelope messageEnvelope(uptime, handler, message);
        mMessageEnvelopes.insertAt(messageEnvelope, i, 1);
        // 如果当前正在发送消息，那么不再调用 wake()，直接返回。
        if (mSendingMessage) {
            return;
        }
    } // 释放锁
    
    // 当把消息加入到消息队列的头部时，需要唤醒 poll 循环。
    if (i == 0) {
        wake();
    }
}
```

---

### 2.6 小结
#### nativeInit()方法
- 创建了 NativeMessageQueue 对象，增加其引用计数，并将 NativeMessageQueue 指针 mPtr 保存在 Java 层的 MessageQueue
- 创建了 Native Looper 对象
- 调用 epoll 的 epoll_create()/epoll_ctl() 来完成对 mWakeEventFd 和 mRequests 的可读事件监听

#### nativeDestroy() 方法
- 调用 RefBase::decStrong() 来减少对象的引用计数
- 当引用计数为 0 时，则删除 NativeMessageQueue 对象

#### nativePollOnce() 方法
- 调用 Looper::pollOnce() 来完成，空闲时停留在 epoll_wait() 方法，用于等待事件发生或者超时 

#### nativeWake() 方法
- 调用 Looper::wake() 来完成，向管道 mWakeEventfd 写入字符

---

## Native 结构体和类
### 3.1 Message结构体
```
struct Message {
    Message() : what(0) { }
    Message(int what) : what(what) { }
    int what;  // 消息类型
};
```

### 3.2 消息处理类
- MessageHandler 类

```
class MessageHandler : public virtual RefBase {
protected:
    virtual ~MessageHandler() { }
public:
    virtual void handleMessage(const Message& message) = 0;
};
```

- WeakMessageHandler 类，继承于 MessageHandler 类

```
class WeakMessageHandler : public MessageHandler {
protected:
    virtual ~WeakMessageHandler();
public:
    WeakMessageHandler(const wp<MessageHandler>& handler);
    virtual void handleMessage(const Message& message);
private:
    wp<MessageHandler> mHandler;
};

void WeakMessageHandler::handleMessage(const Message& message) {
    sp<MessageHandler> handler = mHandler.promote();
    if (handler != NULL) {
        handler->handleMessage(message); //调用MessageHandler类的处理方法()
    }
}
```

### 3.3 回调类
```
class LooperCallback : public virtual RefBase {
protected:
    virtual ~LooperCallback() { }
public:
    //用于处理指定的文件描述符的poll事件
    virtual int handleEvent(int fd, int events, void* data) = 0;
};
```

### 3.4 Looper类
```
static const int EPOLL_SIZE_HINT = 8;    // 每个 epoll 实例默认的文件描述符个数
static const int EPOLL_MAX_EVENTS = 16;  // 轮询事件的文件描述符的个数上限
```

---

## 总结
- MessageQueue 通过 mPtr 变量保存 NativeMessageQueue 对象，从而使得 MessageQueue 成为 Java 层和 Native 层的枢纽，既能处理上层消息，也能处理 native 层消息；

### 图解
- 红色虚线关系：Java 层和 Native 层的 MessageQueue 通过 JNI 建立关联，彼此之间能相互调用，搞明白这个互调关系，也就搞明白了 Java 如何调用 C++ 代码，C++ 代码又是如何调用 Java 代码
- 蓝色虚线关系：Handler/Looper/Message 这三大类 Java 层与 Native 层并没有任何的真正关联，只是分别在 Java 层和 Native 层的 handler 消息模型中具有相似的功能。都是彼此独立的，各自实现相应的逻辑
- WeakMessageHandler 继承于 MessageHandler 类，NativeMessageQueue 继承于 MessageQueue 类

![image](handler2_page6.png)

---